In [1]:
import streamlit as st
import time
from datetime import datetime as dt
from typing import Dict, List, Optional
import redis
import json
import uuid
import sys
from tinydb import TinyDB, Query
from datetime import datetime as dt, timedelta
import os
from typing import List, Tuple, Optional, Dict, Any
import importlib
import uuid

In [2]:
root_path = "/Users/zejunzheng/Documents/GenAI/stock_searcher"  # Absolute path
sys.path.append(root_path)

In [3]:
import redis_q.redisUtils as rq # import RQJobQ
import dbutils.db_classes as dbs
import webUtils.newsUtils as news_cls
import config.conf as configure
import LLMUtils.llmTools as llm_tools

In [4]:
raw_html_folder = configure.paths['raw_html_dir']
charDB_path = configure.paths['chart_db']
newsDB_path = configure.paths['news_db']
jobDB_path = configure.paths['job_db']
news_downloader = news_cls.TickerCrawler(raw_html_folder, charDB_path, newsDB_path)

In [5]:
job_queue = rq.RQJobQ()

In [1]:
while True:
    try:
        job_id, ticker = job_queue.get_crawler_job()
        
        if job_id:
            print(f"Processing job {job_id} - {ticker}")
            status, url_keys, org = await news_downloader.find_or_download_news_urls(ticker, job_id)
            print(f"Completed job {job_id} - {status}")
            print(url_keys)
            
            print("push job to next step -summarisor")
            job_queue.push_summary_job(job_id, url_keys, ticker, org)
            print("sumary jobs: ", job_queue.RQ.llen(job_queue.sumary_queue_name))
      
    except redis.exceptions.TimeoutError:
        continue
    except Exception as e:
        print(f"Error processing job: {e}")
        time.sleep(1) 

In [6]:
print("sumary jobs: ", job_queue.RQ.llen(job_queue.crawl_queue_name))

sumary jobs:  0
